Data importation:
We import the packages needed and the datasets to use

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
categories_df=pd.read_json("categories.json")
products=pd.read_json("products.json")

In [2]:
categories_df.head(5)

,id,name,path,subCategories
0,abcat0010000,Gift Ideas,"[{'id': 'abcat0010000', 'name': 'Gift Ideas'}]","[{'id': 'pcmcat140000050035', 'name': 'Capturi..."
1,abcat0020001,Learning Toys,"[{'id': 'abcat0010000', 'name': 'Gift Ideas'},...",[]
2,abcat0020002,DVD Games,"[{'id': 'abcat0010000', 'name': 'Gift Ideas'},...",[]
3,abcat0020004,Unique Gifts,"[{'id': 'abcat0010000', 'name': 'Gift Ideas'},...",[]
4,abcat0100000,TV & Home Theater,"[{'id': 'abcat0100000', 'name': 'TV & Home The...","[{'id': 'abcat0101000', 'name': 'TVs'}, {'id':..."


In [3]:
products.head(5)

,sku,name,type,price,upc,category,shipping,description,manufacturer,model,url,image
0,43900,Duracell - AAA Batteries (4-Pack),HardGood,5.49,41333424019,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Compatible with select electronic devices; AAA...,Duracell,MN2400B4Z,http://www.bestbuy.com/site/duracell-aaa-batte...,http://img.bbystatic.com/BestBuy_US/images/pro...
1,48530,Duracell - AA 1.5V CopperTop Batteries (4-Pack),HardGood,5.49,41333415017,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Long-lasting energy; DURALOCK Power Preserve t...,Duracell,MN1500B4Z,http://www.bestbuy.com/site/duracell-aa-1-5v-c...,http://img.bbystatic.com/BestBuy_US/images/pro...
2,127687,Duracell - AA Batteries (8-Pack),HardGood,7.49,41333825014,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Compatible with select electronic devices; AA ...,Duracell,MN1500B8Z,http://www.bestbuy.com/site/duracell-aa-batter...,http://img.bbystatic.com/BestBuy_US/images/pro...
3,150115,Energizer - MAX Batteries AA (4-Pack),HardGood,4.99,39800011329,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,4-pack AA alkaline batteries; battery tester i...,Energizer,E91BP-4,http://www.bestbuy.com/site/energizer-max-batt...,http://img.bbystatic.com/BestBuy_US/images/pro...
4,185230,Duracell - C Batteries (4-Pack),HardGood,8.99,41333440019,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Compatible with select electronic devices; C s...,Duracell,MN1400R4Z,http://www.bestbuy.com/site/duracell-c-batteri...,http://img.bbystatic.com/BestBuy_US/images/pro...


We will set the features to be the X from now and the labels to be the y

In [4]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

new_category_list=[]
for x in products.category:
    new_category_list.append(list(x[-1].values())[1])

products.category=new_category_list
y = products.category
enc = OrdinalEncoder(dtype='int')
y = np.array(y)
enc.fit(y.reshape(-1,1))
aux = enc.transform(y.reshape(-1,1))
aux=pd.DataFrame(aux)

new_categories=[]
threshold=32
values=aux.value_counts().values
indexes=aux.value_counts().index

for x,y in zip(indexes,values):
    if y > threshold:
        new_categories.append(x[0])
    else:
        new_categories.append('other')

new_y=[]
for category in aux.values:
    if category[0] in new_categories:
        new_y.append(category[0])
    else:
        new_y.append('other')

X = products.drop(['category'], axis=1)
y = new_y
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)

Now we create some classes to be used as esential parts in a future pipeline

In [5]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/facurg96/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/facurg96/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/facurg96/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from gensim.models import Word2Vec, KeyedVectors, FastText, Doc2Vec
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from text_normalizer import normalize_corpus


class Word2VecProcessor(TransformerMixin):
    def fit(self, X, y=None):
        model_w2v = Word2Vec(X, vector_size=100, workers=6, epochs=50, min_count=1)  
        model_w2v.save('w2vec_model.txt')
        
        return self

    def transform(self, X):
        model = Word2Vec.load('w2vec_model.txt')
        corpus_vectors=[]
        for element in X:
            main_vector=np.zeros([100,],)
            for word in element:
                sub_vector=model.wv[word]
                main_vector=main_vector+ sub_vector
            main_vector=main_vector/len(element)
            corpus_vectors.append(main_vector)
        return corpus_vectors

class FastTextProcessor(TransformerMixin):
    def fit(self, X, y=None):
        model_ft = FastText(X, vector_size=100, workers=6, epochs=50, min_count=1)  
        model_ft.save('fasttext_model.txt')
        
        return self

    def transform(self, X):
        model = FastText.load('fasttext_model.txt')
        corpus_vectors=[]
        for element in X:
            main_vector=np.zeros([100,],)
            for word in element:
                sub_vector=model.wv[word]
                main_vector=main_vector+ sub_vector
            main_vector=main_vector/len(element)
            corpus_vectors.append(main_vector)
        return corpus_vectors


class DropColumns(TransformerMixin):
    """This Transformer, receives a list of column names
    as a parameter, which are the columns to be dropped."""
    def __init__(self, cols_to_drop):
        self.cols_to_drop = cols_to_drop
    def transform(self, X):
        return X.drop(self.cols_to_drop, axis=1)

    def fit(self, X, y=None):
        return self

class ColumnExtractor(TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    def transform(self, X):
        return X[self.columns]

    def fit(self, X, y=None):
        return self
class MyOneHotEncoding(TransformerMixin):
    def transform(self, X):
        return pd.DataFrame(self.ohe.transform(X), columns = self.ohe_categories)

    def fit(self, X, y=None):
        self.ohe = OneHotEncoder(sparse=False).fit(X)
        self.ohe_categories = self.ohe.categories_
        return self 
        
class NameDescriptionImputation(TransformerMixin):
    def transform(self, X):
        #X["name"] = X["name"].fillna(X["description"])
        X["name"] = X["name"].fillna("UNK")
        #X["description"] = X["description"].fillna("UNK")
        return X

    def fit(self, X_param, y=None):
        return self

class NameDescriptionNormalization(TransformerMixin):
    def transform(self, X):
        X["name"] = normalize_corpus(list(X["name"]))
        #X["description"] = normalize_corpus(list(X["description"]))
        return X

    def fit(self, X, y=None):
        return self

class TfidfVectorizerTransformer(TransformerMixin):
    def transform(self, X):
        return self.tfidf.transform(list(X.iloc[:,0]))

    def fit(self, X, y=None):
        self.tfidf = TfidfVectorizer(norm=None).fit(list(X.iloc[:,0]))
        return self

class OrdinalEncoding(TransformerMixin):
    def fit(self, X, y=None):
        self.enc = OrdinalEncoder(dtype='int')
        self.enc.fit(X)
        return self
    
    def transform(self, X):
        aux = []
        X = np.array(X)
        X = X.reshape(-1,1)
        aux = self.enc.transform(X)
        #aux.value_counts()
        return aux

/usr/lib/python3/dist-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography import x509


In [7]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
import pickle

# Transformers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

# Modeling Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, confusion_matrix, classification_report
from IPython.display import display, Markdown

# Pipelines
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

#liraries for NLP
import os
import sys
# I had some issues importing files
# This is how I addressed them:
cwd = os.getcwd()
add = "/".join(cwd.split("/")[:-1])
sys.path.append(add)

from text_normalizer import normalize_corpus
from sklearn.feature_extraction.text import TfidfVectorizer


COLS_TO_DROP = ['sku','upc', 'shipping','manufacturer', 'model', 'url', 'image']
pipe = Pipeline([
    ('drop columns', DropColumns(COLS_TO_DROP)),
    ('features', FeatureUnion([
        ('price', Pipeline([
            ('extract', ColumnExtractor(["price"])),
            ('imputation', SimpleImputer(strategy="mean")),
            ('scaler', StandardScaler())
        ])),
        ('type', Pipeline([
            ('extract', ColumnExtractor(["type"])),
            ('imputation', SimpleImputer(strategy="most_frequent")),
            ('ohe', OneHotEncoder(drop='first'))
        ])),
        ('name and description', Pipeline([
            ('extract', ColumnExtractor(["name",])),#"description"])),
            ('imputation', NameDescriptionImputation()),
            ('text normalizer', NameDescriptionNormalization()),    
            ('text vectorizer', TfidfVectorizerTransformer())
        ]))
    ]))
])


pipe_name = Pipeline([
        ('name and description', Pipeline([
            ('extract', ColumnExtractor(["name",])),#"description"])),
            ('imputation', NameDescriptionImputation()),
            ('text normalizer', NameDescriptionNormalization()),    
            #('text word2vec', Word2VecProcessor()) #('text vectorizer', TfidfVectorizerTransformer())
            ('text vectorizer', TfidfVectorizerTransformer())
        ]))
])

In [8]:
pipe.fit_transform(X_train, y_train)

X_train=pipe.transform(X_train)
X_test=pipe.transform(X_test)

/home/facurg96/.local/lib/python3.6/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
/home/facurg96/.local/lib/python3.6/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
/home/facurg96/.local/lib/python3.6/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


In [24]:
from sklearn.ensemble import RandomForestClassifier
import wandb

labels=list(enc.get_params())
#wandb.init(project="my-test-project")

cl_rf = RandomForestClassifier(n_jobs=-1)
cl_rf.fit(X_train, y_train)
y_preds = cl_rf.predict(X_test)
y_probas=cl_rf.predict_proba(X_test)
f1_score(y_test, y_preds, average="weighted")
micro=f1_score(y_test, y_preds, average=None)

#wandb.sklearn.plot_classifier(cl_rf, X_train, X_test, y_train, y_test, y_preds, y_probas,labels=labels, model_name="SVC", feature_names=None)

In [25]:
micro

array([0.87747036, 0.71428571, 0.91089109, 0.94117647, 0.77165354,
       0.94736842, 0.96190476, 0.92857143, 0.84444444, 1.        ,
       0.55384615, 0.68387097, 0.77777778, 0.72727273, 0.7826087 ,
       0.5       , 0.96551724, 0.96296296, 0.96551724, 0.95      ,
       0.8440367 , 0.84745763, 0.90909091, 0.6875    , 0.33333333,
       0.89655172, 0.71428571, 0.99341486, 1.        , 0.76190476,
       0.75862069, 0.9442623 , 0.87804878, 1.        , 1.        ,
       0.9       , 0.82352941, 0.88      , 0.84615385, 0.70588235,
       0.81818182, 0.96551724, 0.92      , 0.70588235, 0.66666667,
       0.47058824, 0.88235294, 0.76923077, 1.        , 0.89655172,
       0.5       , 0.90410959, 0.89583333, 0.92682927, 0.81632653,
       1.        , 0.91428571, 0.85236769, 0.33333333, 0.69767442,
       1.        , 0.36363636, 0.70588235, 0.96969697, 0.66666667,
       0.71111111, 0.9       , 0.83870968, 0.89411765, 0.88      ,
       0.76923077, 0.95172414, 0.93333333, 0.73684211, 0.83333

In [178]:
import xgboost as xgb

bst = xgb.XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='multi:softprob')
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)
f1_score(y_test, preds, average="weighted")
micro = f1_score(y_test, preds, average="micro")
sns.distplot(micro)

/home/facurg96/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:04:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.23383353580705116

In [17]:
from sklearn.neighbors import KNeighborsClassifier
KNN_model = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
KNN_pred = KNN_model.predict(X_test)  
f1_score(y_test, KNN_pred, average="weighted")

0.7376066207882696

In [20]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC(max_iter=3000).fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)  
f1_score(y_test, svm_pred, average="weighted")

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.794054196554047